In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import files

# ⚡ 1️⃣ دریافت و استخراج فایل مدل
#uploaded = files.upload()  # آپلود فایل فشرده مدل
MODEL_ZIP_PATH = "centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz"
MODEL_DIR = "centernet_mobilenetv2_fpn_od"
SAVED_MODEL_PATH = os.path.join(MODEL_DIR, "saved_model")
#SAVED_MODEL_PATH = "saved_model"

# 🗂️ استخراج فایل فشرده مدل اگر از قبل استخراج نشده باشد
if not os.path.exists(SAVED_MODEL_PATH):
    print("📦 در حال استخراج مدل...")
    !tar -xvf {MODEL_ZIP_PATH}
else:
    print("✅ مدل از قبل استخراج شده است.")

# 🚀 بارگذاری مدل
print("🚀 در حال بارگذاری مدل...")
model = tf.saved_model.load(SAVED_MODEL_PATH)
print("✅ مدل بارگذاری شد!")

# ⚡ 2️⃣ دریافت و استخراج داده تست
#uploaded = files.upload()
!unrar x test.rar
TEST_PATH = "test"

# 📌 نمایش محتوای فولدر تست
print("\n📂 محتوای test:")
!ls -l test

# 📏 اندازه تصاویر
img_size = (320, 320)

# 📌 3️⃣ بارگذاری تصاویر تست
def load_test_images(folder):
    images = []
    image_paths = []
    for file in os.listdir(folder):
        if file.endswith(".jpg"):
            img_path = os.path.join(folder, file)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is None:
                print(f"⚠️ تصویر بارگذاری نشد: {img_path}")
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_resized = cv2.resize(img, img_size)
            images.append(img_resized / 255.0)
            image_paths.append(img_path)
    print(f"✅ تعداد تصاویر تست: {len(images)}")
    return np.array(images), image_paths

X_test, image_paths = load_test_images(TEST_PATH)

# 📌 4️⃣ تشخیص اشیا در تصاویر تست
def detect_objects(model, images, image_paths):
    detect_fn = model.signatures["serving_default"]  # گرفتن متد مناسب
    for i, img in enumerate(images):
        input_tensor = tf.convert_to_tensor([img], dtype=tf.float32)
        detections = detect_fn(input_tensor)

        # استخراج اطلاعات باندینگ باکس
        print(detections)
        boxes = detections['detection_boxes'][0].numpy()
        scores = detections['detection_scores'][0].numpy()
        classes = detections['detection_classes'][0].numpy().astype(int)

        # 📌 فیلتر کردن تشخیص‌های معتبر (مثلاً فقط بالای 50٪)
        threshold = 0.5
        valid_indices = np.where(scores > threshold)[0]

        # 📌 رسم باندینگ باکس‌ها
        img_bgr = cv2.imread(image_paths[i])  # تصویر اصلی را بارگذاری کن
        img_bgr = cv2.resize(img_bgr, img_size)

        for idx in valid_indices:
            y1, x1, y2, x2 = boxes[idx]  # باندینگ باکس نرمال‌شده (0 تا 1)
            h, w, _ = img_bgr.shape
            x1, y1, x2, y2 = int(x1 * w), int(y1 * h), int(x2 * w), int(y2 * h)

            label = f"Class {classes[idx]}: {scores[idx]:.2f}"
            cv2.rectangle(img_bgr, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img_bgr, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 📌 نمایش نتیجه
        plt.figure(figsize=(6, 6))
        plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()

# 🎯 اجرای تشخیص روی داده تست
detect_objects(model, X_test, image_paths)
